<a href="https://colab.research.google.com/github/emmanuelvaie/google_colab/blob/main/process_inscriptions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Get authentication for reading the google sheet

In [2]:
from google.colab import auth
auth.authenticate_user()

import gspread
from oauth2client.client import GoogleCredentials

gc = gspread.authorize(GoogleCredentials.get_application_default())

Read the google sheet and transform it in a dataframe properly


In [3]:
worksheet = gc.open_by_url('https://docs.google.com/spreadsheets/d/1SCruyMO_mncyvIF8kA9HEN2cTabM8ge3Mj2Ia_yG8Uk/edit#gid=464665913').sheet1

# get_all_values gives a list of rows.
rows = worksheet.get_all_values()

import pandas as pd
df_inscriptions = pd.DataFrame.from_records(rows,columns=rows[0])
df_inscriptions = df_inscriptions.drop(0).reindex()
# let's keep only validated files
df_inscriptions = df_inscriptions[df_inscriptions['statut']=='Validé']

In [4]:
# and filter inscription validated before the 10/10 (all before this date should have a licence)
from datetime import datetime
df_inscriptions['already_processed'] = df_inscriptions['date_statut'].apply(lambda x : True if datetime.strptime(x,"%Y-%m-%d %H:%M:%S") < datetime.strptime('2021-10-10 00:00:00',"%Y-%m-%d %H:%M:%S") else False)

In [5]:
df_inscriptions = df_inscriptions[df_inscriptions['already_processed']==False]

detect the malformed naissance columns and change the date format

In [ ]:
df_inscriptions.dropna()
df_inscriptions['naissance'].count()
df_ddl_1 = df_inscriptions[df_inscriptions['naissance']=='0000-00-00']

In [7]:
df_ddl_2 =df_inscriptions[df_inscriptions['naissance']=='']
df_ddl_2

,saison,statut,date_statut,civil,nom,prenom,email,tel,adresse,cp,ville,pays,nationalite,naissance,profession,entreprise,parent1_civil,parent1_nom,parent1_prenom,parent1_email,parent1_tel,parent2_civil,parent2_nom,parent2_prenom,parent2_email,parent2_tel,contact_civil,contact_nom,contact_prenom,contact_email,contact_tel,docteur_nom,docteur_date,handicap,reglement,documentation,benevolat,nager,assurance,type,licence,categorie,garage,optioncomp,montant,reduction,motifreduction,netapayer,paiement_helloasso,virement,cheque,nbcheques,chequesvacances,couponsport,pass92,caution_nom,caution,especes,autre_details,autre,montant_total,already_processed


In [8]:
df_inscriptions = df_inscriptions[df_inscriptions['naissance']!='0000-00-00']
df_inscriptions = df_inscriptions[df_inscriptions['naissance']!='']

In [9]:
from datetime import datetime
df_inscriptions['naissance']= df_inscriptions[df_inscriptions['naissance']!='0000-00-00']['naissance'].\
apply(lambda x : datetime.strptime(x, '%Y-%m-%d').strftime('%d/%m/%Y'))

In [10]:
df_inscriptions['sexe'] = df_inscriptions['civil'].apply(lambda x : 'H' if x == 'Monsieur' else 'F')

In [11]:
df_inscriptions = df_inscriptions[df_inscriptions.columns.drop(list(df_inscriptions.filter(regex='^parent')))]
df_inscriptions = df_inscriptions[df_inscriptions.columns.drop(list(df_inscriptions.filter(regex='^contact')))]
df_inscriptions = df_inscriptions[df_inscriptions.columns.drop(list(df_inscriptions.filter(regex='[a-z]*cheque[a-z]*')))]
df_inscriptions = df_inscriptions[df_inscriptions.columns.drop(['documentation', 'benevolat', 'nager', 'assurance',\
                                                                'caution_nom', 'caution', 'especes', 'autre_details',\
                                                                'montant_total','saison','statut','date_statut','profession',\
                                                                'entreprise','reglement','garage','montant','reduction',\
                                                                'motifreduction','netapayer', 'paiement_helloasso','virement',\
                                                                'couponsport','pass92','autre'])]

In [12]:
from datetime import date,timedelta
yesterday = date.today() - timedelta(days=1)
yester_str = yesterday.strftime('%Y-%m-%d')

In [13]:
df_inscriptions['docteur_date'] = df_inscriptions['docteur_date'].apply(lambda x : yester_str if x =='0000-00-00' else x)

In [14]:
from datetime import datetime
df_inscriptions['docteur_date']= df_inscriptions['docteur_date'].apply(lambda x : datetime.strptime(x, '%Y-%m-%d').strftime('%d/%m/%Y'))

In [15]:
def reprocess_add(input_str):
  input_str = input_str.str.replace('?','  ')
  #input_str = input_str.replace('\n',' ')

In [16]:
df_inscriptions['docteur_nom'] = df_inscriptions['docteur_nom'].str.upper()
df_inscriptions['ville'] = df_inscriptions['ville'].str.upper()
df_inscriptions['adresse'] = df_inscriptions['adresse'].str.upper()
df_inscriptions['email']= df_inscriptions['email'].str.lower()
#df_inscriptions['adresse'] = df_inscriptions['adresse'].apply(lambda x : reprocess_add(x))
#df_inscriptions['adresse'] = df_inscriptions['adresse'].apply(lambda x : reprocess_add(x))

Split the address

In [17]:
df_inscriptions['NumeroVoie'] = df_inscriptions['adresse'].str.split(n=2,expand=True)[0]
df_inscriptions['TypeVoie'] = df_inscriptions['adresse'].str.split(n=2,expand=True)[1]
df_inscriptions['LibelleVoie'] = df_inscriptions['adresse'].str.split(n=2,expand=True)[2]


Read the tab with the licences already taken

In [19]:
worksheet = gc.open_by_url('https://docs.google.com/spreadsheets/d/1SCruyMO_mncyvIF8kA9HEN2cTabM8ge3Mj2Ia_yG8Uk/edit#gid=464665913').worksheet('licences_deja_prises')

# get_all_values gives a list of rows.
rows = worksheet.get_all_values()

import pandas as pd
df_licences = pd.DataFrame.from_records(rows,columns=rows[0])

In [20]:
df_licences = df_licences.drop(0).reindex()

In [21]:
import unicodedata

def remove_accents(input_str):
  # remove simple quote and '-
  input_str = input_str.replace('\'','')
  input_str = input_str.replace('-','')
  nfkd_form = unicodedata.normalize('NFKD', input_str)
  return u"".join([c for c in nfkd_form if not unicodedata.combining(c)])

remove the accent for 1st name and last name


In [22]:
df_licences['NOM'] = df_licences['NOM'].apply(lambda x : remove_accents(x))
df_licences['PRENOM'] = df_licences['PRENOM'].apply(lambda x : remove_accents(x))
df_inscriptions['nom'] = df_inscriptions['nom'].apply(lambda x : remove_accents(x))
df_inscriptions['prenom'] = df_inscriptions['prenom'].apply(lambda x : remove_accents(x))



In [23]:
df_licencies = pd.DataFrame(data=df_licences['NOM'] + '_' + df_licences['PRENOM'] + '_' + df_licences['DATE NAISSANCE'], columns=['key'])
df_licencies.sort_values(by='key', inplace=True)
df_licencies = df_licencies.reset_index()

In [25]:
!pip install jaro-winkler

In [26]:
import jaro
def jaro_distance_words(s, d):
    r = {}
    for i in range(0,len(d)-1,1):
        r[i] = jaro.jaro_metric(s, d[i])
        
    return r

In [27]:
# create a similar key on the inscriptions dataset
df_inscriptions = df_inscriptions.reset_index()
df_inscriptions['key'] = df_inscriptions['nom'] + '_' + df_inscriptions['prenom'] + '_' + df_inscriptions['naissance']
d = df_inscriptions.key.to_dict()

In [29]:
df_licencies['jaro'] = df_licencies['key'].apply(lambda x: jaro_distance_words(x,d))

In [30]:
test_str = 'ALBANEL_GEOFFROY_11/07/1957'
d_jaro = jaro_distance_words(test_str, d)

In [31]:
# sort the dictionnary in a list of pairs (index, distance)
d_jaro_sorted = sorted(d_jaro.items(), key=lambda item: item[1], reverse=True)
# display the 5 firsts
d_jaro_sorted[:3]

[(0, 1.0), (104, 0.7314814814814814), (62, 0.7286340103544404)]

In [32]:
df_licencies['3_similar'] = df_licencies['jaro'].apply(lambda x: list(sorted(x.items(), key=lambda item: item[1], reverse=True))[:3])

In [33]:
d_jaro_sorted[1][0]

104

In [44]:
# keep only the greatest distance
df_licencies['match'] = df_licencies['3_similar'].apply(lambda x : d[x[0][0]] if (x[0][1]> 0.8) else 'N/A')

In [43]:
# check if it works for some names
df_licencies[df_licencies['key'].apply(lambda x :  True if 'CHEMLOUL' in x else False)]

,index,key,jaro,3_similar,match
67,221,CHEMLOUL_DARIA_15/06/2004,"{0: 0.5737918871252204, 1: 0.635, 2: 0.6535507...","[(23, 0.9316666666666666), (22, 0.776296296296...",CHEMLOUL_DARIA_06/12/2004
68,222,CHEMLOUL_LOTFI_20/07/1968,"{0: 0.5737918871252204, 1: 0.6030769230769231,...","[(22, 0.9866666666666667), (23, 0.757777777777...",CHEMLOUL_LOTFI_07/02/1968


An then build a list of licenced persons

In [ ]:
licencies = df_licencies.to_dict('list')['key']

Check who already has a licence


In [ ]:
df_inscriptions['licence_prise'] = df_inscriptions['key'].apply(lambda x : True if x in licencies else False)

Then save the result : only the licence not already taken

In [ ]:
worksheet = gc.open_by_url('https://docs.google.com/spreadsheets/d/1SCruyMO_mncyvIF8kA9HEN2cTabM8ge3Mj2Ia_yG8Uk/edit#gid=464665913').worksheet("Fitogram")

# get_all_values gives a list of rows.
rows = worksheet.get_all_values()

import pandas as pd
df_fito = pd.DataFrame.from_records(rows,columns=rows[8])
df_fito = df_fito[df_fito['Nom']!=''].drop(8).drop(9)
df_fito = df_fito[df_fito['Email']!='']
df_fito['Email']= df_fito['Email'].str.lower()

In [ ]:
l_fito = pd.DataFrame(df_fito['Email']).to_dict('list')['Email']

Then give a status to the Fitogram account by comparing the address mails


In [ ]:
df_inscriptions['is_in_fitogram'] = df_inscriptions['email'].apply(lambda x : True if x in l_fito else False)

In [ ]:
df_renouvellement_fito = df_fito[df_fito['Produit actif'] == 'Licence A']

In [ ]:
df_renouvellement_fito = df_renouvellement_fito[df_renouvellement_fito['Abonnement (Début)'].\
                                                apply(lambda x : datetime.strptime(x, '%d/%m/%Y').date().year)==2020]


In [ ]:
df_renouvellement_fito2 = df_renouvellement_fito[['Prénom','Nom','Email','Anniversaire','Abonnement (Début)']]

In [ ]:
df_inscr = df_inscriptions[['email','nom','prenom']]

In [ ]:
df_inscr = df_inscr.set_index('email')
df_renouvellement_fito2 = df_renouvellement_fito2.set_index('Email')

In [ ]:
df_renouvellement_fito2 = df_renouvellement_fito2.join(df_inscr, on='Email', how='inner')

In [ ]:
try :
  inscription_reprocessed = worksheet.spreadsheet.worksheet('re-processed')
  inscription_reprocessed.clear()
except:
  inscription_reprocessed = worksheet.spreadsheet.add_worksheet(title="re-processed", rows="350", cols="20")

from gspread_dataframe import set_with_dataframe
#set_with_dataframe(inscription_reprocessed, df_inscriptions[df_inscriptions['licence_prise']==False])
set_with_dataframe(inscription_reprocessed, df_inscriptions)

In [ ]:
try :
  inscription_reprocessed = worksheet.spreadsheet.worksheet('Fitogram to change')
  inscription_reprocessed.clear()
except:
  inscription_reprocessed = worksheet.spreadsheet.add_worksheet(title="Fitogram to change", rows="350", cols="20")

from gspread_dataframe import set_with_dataframe
set_with_dataframe(inscription_reprocessed, df_renouvellement_fito2)

In [ ]:
# maintenant le cahier électronique
# 1) on construit un dataset de clé NOM_PRENOM, naissance, genre
# 2) on le compare au dataset persons : NOM_PRENOM, naisssance, genre en faisant un right outer join (existe dans les inscriptions mais pas dans le cahier)


In [ ]:
df_inscr_bis = df_inscriptions[['nom','prenom', 'naissance','sexe']]

In [ ]:
df_inscr_bis['key'] = df_inscr_bis['nom'] + '_' + df_inscr_bis['prenom']

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
df_inscr_bis = df_inscr_bis.set_index('key')

In [ ]:
worksheet = gc.open_by_url('https://docs.google.com/spreadsheets/d/1SCruyMO_mncyvIF8kA9HEN2cTabM8ge3Mj2Ia_yG8Uk/edit#gid=464665913').worksheet("persons")

# get_all_values gives a list of rows.
rows = worksheet.get_all_values()

import pandas as pd
df_persons = pd.DataFrame.from_records(rows,columns=rows[0])
df_persons = df_persons.drop(0)

In [ ]:
df_persons = df_persons[['LastName','FirstName']]

In [ ]:
import unicodedata

def process_text(input_str):
  # remove simple quote and '-
  input_str = input_str.replace('\'','')
  input_str = input_str.replace('-','')
  input_str = input_str.replace(' ','_')
  nfkd_form = unicodedata.normalize('NFKD', input_str)
  return u"".join([c for c in nfkd_form if not unicodedata.combining(c)])

In [ ]:
df_persons['LastName'] = df_persons['LastName'].apply(lambda x : process_text(x))
df_persons['FirstName'] = df_persons['FirstName'].apply(lambda x : process_text(x))
df_persons['FirstName'] = df_persons['FirstName'].str.upper()


In [ ]:
df_persons['key'] = df_persons['LastName'] + '_' + df_persons['FirstName']

In [ ]:
df_persons = df_persons.set_index('key')

In [ ]:
df_join = df_inscr_bis.join(df_persons, on='key', how='left')

In [ ]:
df_join = df_join[df_join['LastName'].isnull()]

In [ ]:
try :
  inscription_reprocessed = worksheet.spreadsheet.worksheet('Cahier elec.')
  inscription_reprocessed.clear()
except:
  inscription_reprocessed = worksheet.spreadsheet.add_worksheet(title="Cahier elec.", rows="350", cols="20")

from gspread_dataframe import set_with_dataframe
set_with_dataframe(inscription_reprocessed, df_join)

In [ ]:
 df_join = df_join.reset_index()

In [ ]:
 df_join = df_join.drop(columns=['key', 'LastName', 'FirstName'])

In [ ]:
df_join = df_join.rename(columns={'nom':'LastName','prenom':'FirstName','sexe':'Gender', 'naissance':'Birthday'})
df_join['NameAffix'] =''


In [ ]:
df_join['Title'] = ''
df_join = df_join[['FirstName','LastName', 'NameAffix','Title','Gender','Birthday']]

In [ ]:
df_join['Gender'] = df_join['Gender'].apply(lambda x : 'féminin' if x=='F' else 'masculin')

In [ ]:
df_join['Birthday'] = df_join['Birthday'].apply(lambda x : str(x).replace('/','.'))

In [ ]:
try :
  inscription_reprocessed = worksheet.spreadsheet.worksheet('Persons to import')
  inscription_reprocessed.clear()
except:
  inscription_reprocessed = worksheet.spreadsheet.add_worksheet(title="Persons to import", rows="100", cols="20")

from gspread_dataframe import set_with_dataframe
set_with_dataframe(inscription_reprocessed, df_join)